In [ ]:
# Mounting Google Drive
from google.colab import drive
from os.path import join
ROOT = '/content/drive'
print(ROOT)
drive.mount(ROOT)

/content/drive
Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/My\ Drive/Colab Notebooks/ProjectData/FaceRecognition

/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition


## Importing the dataset and necessary libraries

In [ ]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
%matplotlib inline

In [23]:
Dataset_path = "dataset2"
face_detection_path= "face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
proto_path = "face_detection_model/deploy.prototxt"
embedding_gen_path = "face_detection_model/openface_nn4.small2.v1.t7"
output_path = "pickle"

In [24]:
print("Loading face detector.....")
detector = cv2.dnn.readNetFromCaffe(proto_path, face_detection_path)

Loading face detector.....


In [25]:
print("Loading face embedder....")
embedder = cv2.dnn.readNetFromTorch(embedding_gen_path);

Loading face embedder....


In [26]:
imagepath = list(paths.list_images(Dataset_path))


In [27]:
imagepath

['dataset2/Deepika_Padukone/104.jpg',
 'dataset2/Deepika_Padukone/123.jpg',
 'dataset2/Deepika_Padukone/118.jpg',
 'dataset2/Deepika_Padukone/1.jpg',
 'dataset2/Deepika_Padukone/106.jpg',
 'dataset2/Deepika_Padukone/117.jpg',
 'dataset2/Deepika_Padukone/114.jpg',
 'dataset2/Deepika_Padukone/111.jpg',
 'dataset2/Deepika_Padukone/12.jpg',
 'dataset2/Deepika_Padukone/109.jpg',
 'dataset2/Deepika_Padukone/126.jpg',
 'dataset2/Deepika_Padukone/105.jpg',
 'dataset2/Deepika_Padukone/108.jpg',
 'dataset2/Deepika_Padukone/112.jpg',
 'dataset2/Deepika_Padukone/11.jpg',
 'dataset2/Deepika_Padukone/10.jpg',
 'dataset2/Deepika_Padukone/119.jpg',
 'dataset2/Deepika_Padukone/124.jpg',
 'dataset2/Deepika_Padukone/121.jpg',
 'dataset2/Deepika_Padukone/110.jpg',
 'dataset2/Deepika_Padukone/125.jpg',
 'dataset2/Deepika_Padukone/120.jpg',
 'dataset2/Deepika_Padukone/101.jpg',
 'dataset2/Deepika_Padukone/115.jpg',
 'dataset2/Deepika_Padukone/102.jpg',
 'dataset2/Deepika_Padukone/128.jpg',
 'dataset2/Deepik

In [28]:
detector_haar = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

In [29]:
def getFace_haar(image):
  max = -1
  x1 = 0
  x2=0
  y1=0
  y2 = 0
  faces = detector_haar.detectMultiScale(image, 1.3,5)
  for (x,y,w,h) in faces:
    print(w,h)
    if w < 20 or h <20:
      continue
    area = h*w
    if max < area:
      max = area
      x1 = x
      y1 = y
      x2 = x+w
      y2 = y+h

    return x1, x2, y1, y2
    

In [30]:
def getFace_CV2DNN(image):
  (h,w) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)),1.0, (300,300),(104.0, 177.0, 123.0), swapRB= False, crop = False)
  detector.setInput(blob)
  detections = detector.forward()
  max = -1
  fHM = 0
  fWM = 0
  x1 = 0
  x2=0
  y1=0
  y2 = 0
  for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2];
    
    if confidence < 0.7:
      continue

    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")
    cv2.rectangle(image, (startX, startY), (endX, endY), (0,0,255), 2)
    
    
    fH = endX - startX
    fW = endY - startY
    print("ggg-",fW,fH)
    if fH < 20 or fW < 20:
      continue
    area = fH*fW
    if max < area:
      max = area
      x1 = startX
      x2 = endX
      y1 = startY
      y2 = endY
  return x1,x2,y1,y2


In [35]:
knownEmbeddings = []
knownNames = []
total = 0
length=len(imagepath)
for (i, imageP) in enumerate(imagepath):
  print("Processing image ",int(i+1),"/",length)
  name = imageP.split(os.path.sep)[-2]
  #print(name,imageP)

  image =  cv2.imread(imageP)
  image = imutils.resize(image, width=600)
  
  '''if(i % 5 == 0):
    plt.imshow(image)
    plt.show()
  
  '''
  cord = getFace_haar(image)
  if cord == None:
    continue
  x1, x2, y1, y2 = cord
  cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)
  face = image[y1:y2, x1:x2]
  try : 
    faceBlob = cv2.dnn.blobFromImage(face, 1/255, (96,96), (0,0,0), swapRB = True, crop= False)
  except:
    print('Some Error With image')
    cv2_imshow(image)
  embedder.setInput(faceBlob)
  vec = embedder.forward()
  knownNames.append(name)
  knownEmbeddings.append(vec.flatten())

  total = total+ 1



  #cv2_imshow(image)



Processing image  1 / 2456
142 142
Processing image  2 / 2456
271 271
Processing image  3 / 2456
Processing image  4 / 2456
211 211
Processing image  5 / 2456
293 293
Processing image  6 / 2456
171 171
Processing image  7 / 2456
73 73
Processing image  8 / 2456
Processing image  9 / 2456
Processing image  10 / 2456
264 264
Processing image  11 / 2456
341 341
Processing image  12 / 2456
Processing image  13 / 2456
309 309
Processing image  14 / 2456
295 295
Processing image  15 / 2456
274 274
Processing image  16 / 2456
291 291
Processing image  17 / 2456
89 89
Processing image  18 / 2456
295 295
Processing image  19 / 2456
429 429
Processing image  20 / 2456
204 204
Processing image  21 / 2456
Processing image  22 / 2456
124 124
Processing image  23 / 2456
Processing image  24 / 2456
274 274
Processing image  25 / 2456
323 323
Processing image  26 / 2456
288 288
Processing image  27 / 2456
Processing image  28 / 2456
262 262
Processing image  29 / 2456
144 144
Processing image  30 / 24

In [36]:
print("Serialising",total, "images and saving it into pickle files")
data = {"name":knownNames, "embedding":knownEmbeddings}
pickcle_path = output_path+"/face_dataset_Caffemodel.pickle"
f = open(pickcle_path, "wb")
f.write(pickle.dumps(data))
f.close()

Serialising 2087 images and saving it into pickle files
